使用记忆系统可以给Agent或者Crew可以赋予一个角色、团队临时性或者经历过的知识，对于团队外部即时的、更加庞大的外部知识，将无法使用，因此给Agent、Crew引入knowledge的概念，以供Agent、Crew在执行Task时，查看这些知识

相比较memory,knowledge有以下优势：

- 类似RAG，可以使用领域内特定的知识
- 即时的知识

让我们深入crewai，看看knowledge的构建、使用

```mermaid
classDiagram 
    BaseKnowledgeSource <|-- BaseFileKnowledgeSource
    BaseKnowledgeSource <|-- StringKnowledgeSource
    BaseFileKnowledgeSource <|-- ExcelKnowledgeSource

    class BaseKnowledgeSource{
        chunk_size:int=4000
        chunk_overlap: int = 200
        chunks: List[str]
        chunk_embeddings: List[np.ndarray]
        model_config
        storage: KnowledgeStorage
        metadata: Dict[str, Any]
        collection_name: Optional[str] 
    
        load_content(): Dict[Path, str]
        add()
        get_embeddings()
        _chunk_text(text: str): List[str]
        save_documents(metadata)
    }
    class BaseFileKnowledgeSource{
        file_path: 
        content: Dict[Path, str]
        storage: KnowledgeStorage
        load_content(): Dict[Path, str]
        save_documents(metadata)
    }
    class StringKnowledgeSource{
        content: str
        collection_name: Optional[str]
        add()
        _chunk_text(text: str): List[str]        
    }
    class ExcelKnowledgeSource{
        load_content()
        add()
        _chunk_text()
    }

```

从代码中可以看出，CrewAI 支持两大类知识源，即：1）文本源,如字符串、文本文件、PDF文件、Markdown文件；2）结构化数据源，如csv、excle、json等

In [1]:
from crewai import Agent,Task,Crew,Process,LLM

llm=LLM(
    model="ollama_chat/qwen2.5:latest", 
    base_url="http://localhost:11434")

embedder={
    "provider": "ollama",
    "config": {
        "model": 'quentinz/bge-large-zh-v1.5:latest',
        "base_url": 'http://localhost:11434'
    }
}

In [6]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

string_source=StringKnowledgeSource(
    content="小伍，正在学习crewai的knowledge部分知识"
)

agent=Agent(
    role="客服",
    goal="以礼貌友好的态度回答用户问题",
    backstory="你是一个经验丰富的客服，总能收获用户的认可",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

task=Task(
    description="回答用户问题：{question}",
    expected_output="回答用户问题",
    agent=agent
)

crew=Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    manager_llm=llm,
    planning_llm=llm,
    process=Process.sequential,
    knowledge_sources=[string_source],
    embedder=embedder
)

result=crew.kickoff(inputs={"question":"小伍在做什么？"})


Overriding of current TracerProvider is not allowed


# Agent: 客服
## Task: 回答用户问题：小伍在做什么？


# Agent: 客服
## Final Answer: 
您好！小伍目前正在专心学习CrewAI的知识库部分内容。他正努力提升自己的知识水平，以便更好地为您服务。如果有任何问题或需要帮助，请随时告诉我！感谢您的耐心等待。




不仅可以给团队设置外部知识，还可以给agent设置外部知识

In [8]:
string_source=StringKnowledgeSource(
    content="小伍，你希望成为一个怎样的人？控制自己能控制的，掌握自己不能控制，更好的物质条件，在每个抉择的十字路口，有更多的选择",
    metadata={"类别":"小伍的期待"}
)

agent=Agent(
    role="customer",  # 注意：这里创建向量数据库使用了该字符串，使用中文时，无法创建
    goal="以礼貌友好的态度回答用户问题",
    backstory="你是一个经验丰富的客服，总能收获用户的认可",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    knowledge_sources=[string_source],
    embedder_config=embedder
)

task=Task(
    description="回答用户问题：{question}",
    expected_output="回答用户问题",
    agent=agent
)

crew=Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)

result=crew.kickoff(inputs={"question":"小伍对未来有什么期待？"})

Overriding of current TracerProvider is not allowed


# Agent: customer
## Task: 回答用户问题：小伍对未来有什么期待？


# Agent: customer
## Final Answer: 
小伍对未来有很多美好的期待。他希望成为一个对社会有所贡献、有责任感的人，并且在追求个人成长的同时，也能关注周围的世界和他人。小伍认为，最重要的是控制自己能控制的方面，比如努力学习、提升自我，同时也学会接受那些无法掌控的事情，比如外部环境的变化或人际关系。

关于更好的物质条件，小伍的目标是通过自己的努力和智慧来实现财务自由，这样不仅可以为自己创造更舒适的生活，也能为家人提供一个更加稳定的支持。在面对人生中的每一个十字路口时，他希望能够有更多的选择权，这不仅意味着有多种可能性可以考虑，更重要的是能够在充分了解各种选项后做出最适合自己的决定。

总的来说，小伍希望通过不懈的努力和明智的决策，实现个人价值的最大化，并且在这个过程中不断成长和进步。




除了以上使用creawai官方的知识源外，还可以自定义知识源，以下例子我们将百度百科的相关词条整理为外部知识源

In [2]:
# 利用https://baike.deno.dev/获取百度百科查询地址
import re
import requests
from bs4 import BeautifulSoup

def get_html(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers, stream=True, timeout=(5,5))
    
    # 检查响应状态码
    if response.status_code == 200:
        return response
    else:
        print(f"请求失败，状态码：{response.status_code}")
        return None

def build_md_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    # 获取summary部分
    md_summary='# 概览\n'
    summary_div= soup.find_all('div', class_='J-summary')  # 替换为实际的数据选择器
    summary_paragraphs = summary_div[0].find_all('div', {'data-tag': 'paragraph'})
    for paragraph in summary_paragraphs:
        text = paragraph.get_text(strip=True)
        text=re.sub(r'\[[1-9][0-9]*\]','',text)
        md_summary+=text+'\n\n'
    
    # 获取内容部分
    md_content=''
    content_div = soup.find_all('div', class_='J-lemma-content')  # 替换为实际的数据选择器
    paragraphs = content_div[0].find_all('div', {'data-tag': ['paragraph','header']})
    for paragraph in paragraphs:
        data_tag=paragraph['data-tag']
        text = paragraph.get_text(strip=True)

        if data_tag=='header':
            text=text.replace('播报编辑','')
            data_level=int(paragraph['data-level'])
            text='#'*data_level+' '+text+'\n'
        else:
            text=re.sub(r'\[[1-9][0-9]*\]','',text)
            text+='\n\n'
        
        md_content+=text

    # 将结果写入文件
    return md_summary,md_content


In [3]:
from crewai import Agent,Task,Crew,Process,LLM
from crewai.knowledge.source.base_file_knowledge_source import BaseKnowledgeSource

from typing import Dict,Any,List
from pydantic import BaseModel,Field

class BaiKeKonwledgeSource(BaseKnowledgeSource):
    """从百度百科构建知识源"""
    keywords:List[str]=Field(description="用于百科查询的关键词")

    def validate_content(self) -> Dict[Any, str]:
        """获取百度百科知识内容"""
        # 收集百科返回文本
        knowledge_list=[]
        for keyword in self.keywords:
            print('fetch content by keyword',keyword)
            url = f"https://baike.deno.dev/item/{keyword}"
            response = get_html(url).json()
            baike_url=response['data']['link']
            print('baike url:',baike_url)

            md_summary,md_content=build_md_from_html(get_html(baike_url).text)
            article=f"""
                Title:{keyword}
                URL:{baike_url}
                Summary:{md_summary}
                Content:{md_content}
                """
            knowledge_list.append(article)
        
        # 聚合所有文本
        knowledge_content='||'.join(knowledge_list)

        return {'baike':knowledge_content}


    def add(self) -> None:
        """处理和存储百科内容"""
        content = self.validate_content()
        for _, text in content.items():
            chunks = self._chunk_text(text)
            self.chunks.extend(chunks)

        self._save_documents()

keywords=["自动驾驶系统", "北京市自动驾驶汽车条例","自动驾驶路测牌照"]
baike_knowledge_source=BaiKeKonwledgeSource(
    keywords=keywords,
    chunk_size=512,
    chunk_overlap=30)


In [4]:
# 这段代码有个问题，当把知识赋予agent时，其role必须是英文，导致输出必定英文，目前将知识赋予团队解决
auto_driving_expert=Agent(
    role="自动驾驶专家",
    goal="普及自动驾驶知识，向广大群众推广自动驾驶",
    backstory="在头部汽车制造公司担任市场部总监多年，熟悉自动驾驶的原理，风险",
    llm=llm
)


discuss_task=Task(
    description="发表一篇关于开放L3级别自动驾驶演讲，讲明白L3级别自动驾驶的定义，开展测试的注意事项，违法交通规则的处理措施等",
    agent=auto_driving_expert,
    expected_output="以中文输出，每个要点输出一段文字，不少于300字，输出为markdow格式"
)

crew=Crew(
    agents=[auto_driving_expert],
    tasks=[discuss_task],
    verbose=True,
    process=Process.sequential,
    language='Chinese',
    knowledge_sources=[baike_knowledge_source],
    embedder=embedder
)

result=crew.kickoff()

fetch content by keyword 自动驾驶系统
baike url: https://baike.baidu.hk/item/%E8%87%AA%E5%8A%A8%E9%A9%BE%E9%A9%B6%E7%B3%BB%E7%BB%9F/22031260
fetch content by keyword 北京市自动驾驶汽车条例
baike url: https://baike.baidu.hk/item/%E5%8C%97%E4%BA%AC%E5%B8%82%E8%87%AA%E5%8A%A8%E9%A9%BE%E9%A9%B6%E6%B1%BD%E8%BD%A6%E6%9D%A1%E4%BE%8B/65010072
fetch content by keyword 自动驾驶路测牌照
baike url: https://baike.baidu.hk/item/%E8%87%AA%E5%8A%A8%E9%A9%BE%E9%A9%B6%E8%B7%AF%E6%B5%8B%E7%89%8C%E7%85%A7/22583909
# Agent: 自动驾驶专家
## Task: 发表一篇关于开放L3级别自动驾驶演讲，讲明白L3级别自动驾驶的定义，开展测试的注意事项，违法交通规则的处理措施等


# Agent: 自动驾驶专家
## Final Answer: 
# 开放L3级别自动驾驶的定义与开展测试注意事项

## 定义
开放L3级别的自动驾驶是指车辆能够在某些特定条件下，在驾驶员监督下自动完成驾驶任务。在这些情况下，系统能够处理所有驾驶任务，并且在必要时可以警告或干预。然而，当车辆提示需要人工接管操作时，驾驶者必须迅速反应并控制车辆。

## 开展测试注意事项

### 法律法规遵从
开展L3级别自动驾驶的测试需遵守当地交通法律法规。例如，在中国，企业需获得相关部门颁发的道路测试牌照及试点资质，并满足一系列的安全评估条件。这包括但不限于具备运行运营能力、责任承担能力、网络安全和数据安全相关保障能力。

### 安全性要求
确保车辆系统在各种环境下的安全性是关键。在开始测试前，必须通过严格的安全评估并符合车辆运营管理的要求。此外，还需确定安全员及其他相关人员的资格条件，并制定完善的运营方案及应急处理机制。

### 人员配置与培训
需要配备经过专业训练的安全